# **Réseaux de transports à la Réunion**

In [1]:
import pandas as pd
from google.transit import gtfs_realtime_pb2
import requests

## **Parsing GTFS real-time TripUpdate messages**

### TripDescriptor

In [2]:
def parse_feed_trip(feed_entity):
    trips = dict()
    trip = {
        "trip_id" : feed_entity.trip_update.trip.trip_id,
        "route_id" : feed_entity.trip_update.trip.route_id,
        "direction_id" : feed_entity.trip_update.trip.direction_id,
        "start_time" : feed_entity.trip_update.trip.start_time,
        "start_date" : feed_entity.trip_update.trip.start_date,
        "schedule_relationship" : feed_entity.trip_update.trip.schedule_relationship
    }
    trips[entity.trip_update.trip.trip_id] = trip
    return (trip)
    
    

### VehicleDescriptor

In [3]:
def parse_feed_vehicle(entity):
    vehicle = {
        "vehicle_id" : entity.trip_update.vehicle.id,
        "vehicle_label" : entity.trip_update.vehicle.label,
        "vehicle_license_plate " : entity.trip_update.vehicle.license_plate
    }
    return (vehicle)

### StopTimeUpdate 

## **Parsing GTFS VehiclePosition messages**

### Position

In [4]:
def parse_feed_position(entity):
    position = {
        "latitude" : entity.vehicle.position.latitude,
        "longitude" : entity.vehicle.position.longitude,
        "bearing" : entity.vehicle.position.bearing
    }
    return (position)

## Position en temps réele des bus alterneo 

In [9]:
vehicles = dict()
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('https://proxy.transport.data.gouv.fr/resource/alterneo-civis-gtfs-rt-vehicle-position')

feed.ParseFromString(response.content)
for entity in feed.entity:
    if entity.HasField('vehicle'):
        vehicle = {
            "position":parse_feed_position(entity)
        }
    vehicles[entity.id] = vehicle

In [10]:
all_vehicles = [v for v in vehicles.values()]

In [11]:
vehicles_position = pd.DataFrame([v.get("position") for v in all_vehicles])
vehicles_position.head(5)

,latitude,longitude,bearing
0,-21.306221,55.426483,146.0
1,-21.341715,55.473049,200.0
2,-21.325361,55.446918,303.0
3,-21.175911,55.455757,345.0
4,-21.264576,55.437824,305.0


In [12]:
import folium
from folium.plugins import MarkerCluster

map = folium.Map([-21.34,55.56], zoom_start = 13)

# Ajout des positions trouvées sur la carte
def create_folium_marker(x):
    return folium.Marker(x)

folium_markers = [create_folium_marker((lat,long)) for lat,long in zip(vehicles_position["latitude"],vehicles_position["longitude"])]

for fm in folium_markers:
    fm.add_to(map)
map